Load Dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from scipy.stats import zscore

# Step 1: Load the dataset
df = pd.read_csv('diabetes_dataset.csv')  # Make sure the dataset file is in the same folder or specify the full path

# Step 2: Remove outliers using Z-score
# Calculate the Z-scores and filter out rows where any Z-score > 3 or < -3 for numerical columns
numerical_cols = df.select_dtypes(include=[np.number]).columns
z_scores = np.abs(zscore(df[numerical_cols]))
df = df[(z_scores < 3).all(axis=1)]  # Keep rows without outliers

# Step 3: Encode categorical columns (text data) to numerical
# Using LabelEncoder for binary columns and get_dummies for multi-category columns
for column in df.select_dtypes(include='object').columns:
    if df[column].nunique() == 2:
        df[column] = LabelEncoder().fit_transform(df[column])
    else:
        df = pd.get_dummies(df, columns=[column], drop_first=True)  # Create dummy variables for categorical columns

# Step 4: Separate features (X) and target label (y), then scale the features
X = df.drop(columns='target')  # Assume 'target' is the column we want to predict
y = df['target']
X_scaled = StandardScaler().fit_transform(X)  # Standardize the feature columns

# Step 5: Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Step 6: Initialize models and train each one, checking accuracy
models = {
    'SVM': SVC(),
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier()
}

print("Model Accuracies (before PCA):")
for model_name, model in models.items():
    model.fit(X_train, y_train)  # Train the model
    y_pred = model.predict(X_test)  # Predict on test data
    accuracy = accuracy_score(y_test, y_pred)  # Calculate accuracy
    print(f"{model_name}: {accuracy:.2f}")

# Step 7: Apply PCA for dimensionality reduction
# Use PCA to reduce the features while retaining 95% of the explained variance
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

# Split the PCA-transformed data into training and testing sets
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, test_size=0.2, random_state=42)

print("\nModel Accuracies (after PCA):")
for model_name, model in models.items():
    model.fit(X_train_pca, y_train_pca)  # Train the model on PCA-transformed data
    y_pred_pca = model.predict(X_test_pca)  # Predict on test data
    accuracy_pca = accuracy_score(y_test_pca, y_pred_pca)  # Calculate accuracy
    print(f"{model_name} with PCA: {accuracy_pca:.2f}")

Remove Outliers Using Z-score
